In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow==1.15
!pip install keras==2.2.5
!pip install git+https://www.github.com/keras-team/keras-contrib.git

     |████████████████████████████████| 412.3MB 28kB/s 
     |████████████████████████████████| 3.8MB 18.9MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 512kB 37.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=f664f133c40ae89bf6e5fdf98753133b38f07bf8673acf9068520857b909fab9
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [3]:
from keras.models import Sequential, Model, Input
from keras.layers import LSTM, Dense, TimeDistributed, Activation, Bidirectional, Masking, Embedding, Dropout, Flatten, concatenate, Conv1D, MaxPool1D, Lambda, Softmax
from keras import backend as K
from keras_contrib.layers import CRF
from keras.utils import plot_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_contrib.losses import  crf_loss
from keras_contrib.metrics import crf_accuracy

Using TensorFlow backend.


In [4]:
from keras.models import load_model
m = load_model("/content/drive/MyDrive/model/model-1.h5")






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [6]:
import numpy as np
from numpy import argmax

In [7]:
char_encode = np.loadtxt('/content/drive/MyDrive/fast_text/3 address/test/char_encode.txt')
print(char_encode.shape)
char_encode = char_encode.reshape(int(char_encode.shape[0]/42), 42, char_encode.shape[1])
print(char_encode.shape)

(32298, 32)
(769, 42, 32)


In [8]:
# chuyển từ char encode sang char embedding
LEN_OF_VOCAB = 137
shape = char_encode.shape
char_embedd = np.zeros([shape[0],shape[1],shape[2],LEN_OF_VOCAB])
for i in range(shape[0]):
  for j in range(shape[1]):
    for k in range(shape[2]):
      char_int = char_encode[i,j,k]
      char_int = char_int.astype(np.int64)
      onehot = np.zeros(LEN_OF_VOCAB)
      onehot[char_int] = 1
      char_embedd[i,j,k,:] = onehot

print(char_embedd.shape)

(769, 42, 32, 137)


In [9]:
tag = np.loadtxt('/content/drive/MyDrive/fast_text/3 address/test/tag_test.txt')
tag = tag.reshape(int(tag.shape[0]/42), 42, tag.shape[1])
print(tag.shape)

(769, 42, 14)


In [10]:
word = np.loadtxt('/content/drive/MyDrive/fast_text/3 address/test/word_test.txt')
word = word.reshape(int(word.shape[0]/42), 42, word.shape[1])
print(word.shape)

(769, 42, 300)


In [11]:
y_pred = m.predict([char_embedd, word])
y_true = tag
print(y_pred.shape)
print(y_true.shape)

(769, 42, 14)
(769, 42, 14)


In [12]:
from sklearn.metrics import classification_report, confusion_matrix

In [13]:
y_pred = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1], y_pred.shape[2])
print(y_pred.shape)
y_pred = np.argmax(y_pred,axis=1)
print(y_pred.shape)

(32298, 14)
(32298,)


In [14]:
y_true = y_true.reshape(y_true.shape[0]*y_true.shape[1], y_true.shape[2])
print(y_true.shape)
y_true = np.argmax(y_true,axis=1)
print(y_true.shape)

(32298, 14)
(32298,)


In [15]:
from sklearn.metrics import accuracy_score
acc=0
false=0
for i in range(len(y_true)):
  if y_true[i]!=0:
    if y_true[i]==y_pred[i]:
      acc+=1
    else:
      false+=1
print(acc)
print(false)
print('accuracy = {}'.format(acc/(acc+false)) )

8169
86
accuracy = 0.9895820714718353


In [16]:
dict = []
for element in y_true:
  if element not in dict:
    dict.append(element)
print(dict)

cnf_matrix = confusion_matrix(y_true, y_pred, labels=dict)
for i in range(len(cnf_matrix)):
  for j in range(len(cnf_matrix)):
    cnf_matrix[i][j] = round(cnf_matrix[i][j], 3)
print(cnf_matrix)

arr_to_cal_predict = []
for i in range(len(cnf_matrix)):
  tong= 0
  for j in range(len(cnf_matrix)):
    tong+=cnf_matrix[j][i]
  arr_to_cal_predict.append(tong)
print(arr_to_cal_predict)
# lấy độ predict
predict = []
for i in range(len(cnf_matrix)):
  tong=cnf_matrix[i][i]/arr_to_cal_predict[i]
  predict.append(tong)
print(predict)

[1, 2, 3, 4, 5, 6, 7, 0]
[[  882    38     0     0     0     0     0     0]
 [   48  2895     0     0     0     0     0     0]
 [    0     0  1316     0     0     0     0     0]
 [    0     0     0   769     0     0     0     0]
 [    0     0     0     0   769     0     0     0]
 [    0     0     0     0     0   769     0     0]
 [    0     0     0     0     0     0   769     0]
 [    0     0     0     0     0     0     0 24043]]
[930, 2933, 1316, 769, 769, 769, 769, 24043]
[0.9483870967741935, 0.9870439822707125, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [17]:
normalized_confusion_matrix = cnf_matrix/cnf_matrix.sum(axis = 1, keepdims = True)
for i in range(len(normalized_confusion_matrix)):
  for j in range(len(normalized_confusion_matrix)):
    normalized_confusion_matrix[i][j] = round(normalized_confusion_matrix[i][j], 3)
print(normalized_confusion_matrix)

[[0.959 0.041 0.    0.    0.    0.    0.    0.   ]
 [0.016 0.984 0.    0.    0.    0.    0.    0.   ]
 [0.    0.    1.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    1.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    1.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    1.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    1.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    1.   ]]
